In [9]:
import math
from typing import Tuple

In [10]:
def normal_approximation_to_binomial(n, p):
    mu = p * n
    sigma = math.sqrt(p * (1 -p) * n)
    return mu, sigma

In [11]:
def normal_cdf(x, mu=0, sigma=1)
    return (1 + math.erf((x -mu) / math.sqrt(2) / sigma)) / 2

In [17]:
normal_probability_below = normal_cdf
# https://en.wikipedia.org/wiki/Cumulative_distribution_function
# When the distribution of a dataset is normal
# The probability the variable is below a given threshold, x
# such as in a normal distribution of 0-10, the probability of given number 5 is 50%

In [13]:
def normal_probability_above(lo, mu, sigma):
    """Just the opposite, the probability of a given number, above that number"""
    return 1 - normal_cdf(lo, mu, sigma)

In [23]:
# can also find probability between numbers
# the probability that a given number is between lo and hi
def normal_probability_between(lo, hi, mu, sigma):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

In [34]:
normal_probability_between(-1, 1, 0, 1)
# a single standard deviation
# in a bell curve, the probability of 0 to -1 and 0 to 1 added together

0.6826894921370859

In [28]:
# or outside numbers
# the probability that a given number is NOT between lo and hi
def normal_probability_outside(lo, hi, mu, sigma):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [36]:
normal_probability_outside(-1, 1, 0, 1)
# everything outside the single standard deviation in a bell curve
# in a bell curve, the probability of 0 to -1 and 0 to 1 added together

0.31731050786291415

In [38]:
# have to use binary search i.e. split everything in half to search for what you want
# repeatedly bisects intervals until it narrows in on a z that's close enough to the desired probability
def inverse_normal_cdf(p, mu = 0, sigma = 1, tolerance = 0.00001):
    if mu !=0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance = tolerance)
    low_z = -10.0                   # normal_cdf(-10) is (very close to) 0)
    hi_z = 10.0                     # normal_cdf(10) is (very close to) 1)
    while hi_z - low_z > tolerance: 
        mid_z = (low_z + hi_z) / 2  # consider the midpoint
        mid_p = normal_cdf(mid_z)   # and the CDF's value there
        if mid_p < p:
            low_z = mid_z           # midpoint too low, search above it
        else:
            hi_z = mid_z            # midpoint too high, search below it
    return mid_z

In [39]:
def normal_upper_bound(probability, mu = 0, sigma=1):
    """Returns the z for which P(Z <= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)
# for a given probability, mu, sigma, we what 

In [58]:
def normal_lower_bound(probability, mu=0, sigma=1):
    """Returns the z for which P(Z >= z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

In [61]:
def normal_two_sided_bounds(probability, mu, sigma):
    """returns the symmetric bounds that 
       contain the specified probability"""
    tail_probability = (1 - probability) / 2
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

In [70]:
normal_two_sided_bounds(.33, 0, 1)

# with a <probability> probability, 
# a mean of the population of <0>, 
# and a standard deviation of <1>,
# The lower and upper bounds will on average be 
# at those points on the normal distribution

(-0.4261493682861328, 0.4261493682861328)

In [72]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [73]:
mu_0, sigma_0

(500.0, 15.811388300841896)

In [75]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [76]:
lower_bound, upper_bound

(469.01026640487555, 530.9897335951244)

* assuming that p really equals 0.5, rejecting the null hypothesis, 
* there is just a 5% chance we observed an X that lies outside this interval, 
* or, 19 times out of 20, this test will give the correct result
* we are safe from type 1 errors, but what about type 2?
* we also want to find the power of the test, not reject the null hypothesis even though it's false,

In [92]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi

526.0073585242053

In [93]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
mu_1, sigma_1

(550.0, 15.732132722552274)

In [94]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
type_2_probability

0.06362051966928273

In [95]:
power = 1 - type_2_probability
power
# the probability that the significance of the test is valid

0.9363794803307173

In [96]:
def two_sided_p_value(x, mu=0, sigma=1):
    """
    How likely are we to see a value at least as extree as x
    in either direction, if our values are from a given number?
    """
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

In [99]:
two_sided_p_value(540.5, mu_0, sigma_0)
# very significant

0.010423777309360283

In [100]:
import random

In [115]:
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # count number of heads
                   for _ in range(1000))              # in 1000 flips
    if num_heads >=  530 or num_heads <= 470:         # and count how often the 
        extreme_value_count += 1                      # num is 'extreme'

In [121]:
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"
print(extreme_value_count)

60


In [124]:
math.sqrt(p * (1 - p) /1000)

0.0061967733539318665

In [130]:
# confidence intervals
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

# We're 95% confident that this interval contains the true parameter p
# which sounds about right since .5 is within that bounds 
# and flipping a coin should have about a 50% chance of being heads or tails

(0.4940490278129096, 0.5559509721870904)

In [148]:
# p-hacking
from typing import List
def run_experiment() -> List[bool]:
    return [random.random() < 0.5 for _ in range(10)]

In [149]:
def reject_fairness(experiment: List[bool]) -> bool:
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [150]:
random.seed(0)
experiments = [run_experiment() for _ in range(10)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

In [151]:
experiments

[[False, False, True, True, False, True, False, True, True, False],
 [False, False, True, False, False, True, False, False, False, False],
 [True, False, False, False, True, True, True, False, False, False],
 [True, False, True, False, False, True, False, True, False, False],
 [True, True, False, True, True, False, True, False, True, False],
 [False, True, True, True, False, False, True, False, False, False],
 [False, False, False, False, False, True, False, True, False, True],
 [True, True, False, False, True, True, False, False, False, False],
 [False, False, False, True, False, True, False, False, False, False],
 [False, False, True, False, False, False, False, True, False, True]]

In [152]:
num_rejections

10